## 載入數據

In [2]:
import pandas as pd
## 把顧客數據load進來
customer_data_df=pd.read_csv("data/Customer_Data.csv")
## 顯示前面十筆數據
customer_data_df.head(10)

,Client_Id,Customer_name,Order_date,Age,Gender,Order_form_status,Monetary_Fish,Monetary_Turtle,Monetary_Elephant
0,1,Tom,2018/5/8,26,M,success,200.0,100.0,0.0
1,1,Tom,2018/5/8,26,M,success,300.0,0.0,800.0
2,1,Tom,2019/6/12,26,M,success,700.0,1000.0,0.0
3,2,Jen,2019/2/11,32,F,success,499.0,299.0,999.0
4,2,Jen,2019/3/12,32,F,fail,0.0,836.0,599.0
5,2,Jen,2019/5/13,32,F,success,600.0,0.0,0.0
6,2,Jen,2020/7/14,32,F,success,400.0,0.0,1200.0
7,3,Ken,2020/5/15,40,M,success,0.0,999.0,999.0
8,3,Ken,2020/5/15,40,M,success,700.0,NaN,600.0
9,4,Louis,2019/2/17,22,F,success,499.0,299.0,699.0


## 檢視訂單的狀態

In [3]:
## 查看訂單狀態有哪些屬性
customer_data_df['Order_form_status'].unique()
## 只保留訂單狀態(Order_form_status)為成功(success)的訂單
customer_data_df = customer_data_df.loc[customer_data_df['Order_form_status'] == 'success', :]
## 查看一下拿掉fail的訂單後，還剩幾筆資料
print('成立的訂單數量: ', len(customer_data_df))

成立的訂單數量:  43


In [4]:
## 查看一下數據資訊
customer_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 48
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Client_Id          43 non-null     int64  
 1   Customer_name      43 non-null     object 
 2   Order_date         43 non-null     object 
 3   Age                43 non-null     int64  
 4   Gender             43 non-null     object 
 5   Order_form_status  43 non-null     object 
 6   Monetary_Fish      42 non-null     float64
 7   Monetary_Turtle    42 non-null     float64
 8   Monetary_Elephant  43 non-null     float64
dtypes: float64(3), int64(2), object(4)
memory usage: 3.4+ KB


In [5]:
## 根據顧客的id來分組，得到他們距離現在最近一次的購買時間點
Recency_df = customer_data_df.groupby('Client_Id')['Order_date'].max().reset_index()
Recency_df

,Client_Id,Order_date
0,1,2019/6/12
1,2,2020/7/14
2,3,2020/5/15
3,4,2020/5/21
4,5,2020/5/25
5,6,2020/7/20
6,7,2020/6/4
7,8,2020/6/9
8,9,2020/6/17
9,10,2020/6/28


In [6]:
## 獲取今天日期
import datetime
now = datetime.date.today()
## 計算距離今天幾天
Recency_df['Recency'] = (pd.to_datetime(now) - pd.to_datetime(Recency_df['Order_date'])).dt.days
## 查看dataframe
Recency_df

,Client_Id,Order_date,Recency
0,1,2019/6/12,408
1,2,2020/7/14,10
2,3,2020/5/15,70
3,4,2020/5/21,64
4,5,2020/5/25,60
5,6,2020/7/20,4
6,7,2020/6/4,50
7,8,2020/6/9,45
8,9,2020/6/17,37
9,10,2020/6/28,26


In [7]:
## 增加一行日期標籤幫助我們接下來的計算，原本Order_date這個欄位會被我們當成是計算次數的欄位
customer_data_df['Date_Tag'] = customer_data_df['Order_date'].astype(str)
## 把同一天消費的訂單合併在一起，等等要忽略掉
dd_Frequency = customer_data_df.groupby(['Client_Id','Date_Tag'])['Order_date'].count().reset_index()
## 計算同一個客人在不同日期消費的次數
Frequency_df = dd_Frequency.groupby('Client_Id')['Order_date'].count().reset_index()
## 重新命名Order_date為Frequency
Frequency_df = Frequency_df.rename(columns={'Client_Id':'Client_Id','Order_date':'Frequency'})

Frequency_df

,Client_Id,Frequency
0,1,2
1,2,3
2,3,1
3,4,4
4,5,3
5,6,5
6,7,3
7,8,6
8,9,5
9,10,7


In [8]:
## 根據顧客ID來計算個別顧客消費Fish品項的總額
Monetary_Fish= pd.DataFrame(customer_data_df.groupby('Client_Id')['Monetary_Fish'].sum().reset_index())
## 重新命名列名
Monetary_Fish = Monetary_Fish.rename(columns={'Client_Id':'Client_Id','Monetary_Fish':'Monetary_sum'})

Monetary_Fish

,Client_Id,Monetary_sum
0,1,1200.0
1,2,1499.0
2,3,700.0
3,4,2897.0
4,5,1498.0
5,6,2199.0
6,7,1198.0
7,8,899.0
8,9,3097.0
9,10,2399.0


In [9]:
## 合併 Monetary 與 Frequency
new_df = pd.merge(Monetary_Fish,Frequency_df, right_on="Client_Id", left_on="Client_Id")
## 計算個別顧客平均消費總額
new_df['Monetary'] = round(new_df['Monetary_sum'] / new_df['Frequency'])
## 合併最後一個欄位Recency
new_customer_data_df = pd.merge(new_df,Recency_df, right_on="Client_Id", left_on="Client_Id")
## 我們只需要RFM所需的數據
new_customer_data_df = new_customer_data_df[['Client_Id','Recency','Frequency', 'Monetary']]
## 顯示新數據
new_customer_data_df

,Client_Id,Recency,Frequency,Monetary
0,1,408,2,600.0
1,2,10,3,500.0
2,3,70,1,700.0
3,4,64,4,724.0
4,5,60,3,499.0
5,6,4,5,440.0
6,7,50,3,399.0
7,8,45,6,150.0
8,9,37,5,619.0
9,10,26,7,343.0


In [10]:
## 幫Recency區分等級
new_customer_data_df['Recency_Level'] = pd.cut(new_customer_data_df['Recency'], bins=[0,40,80,120,160,1000000], labels=[5,4,3,2,1], right=False).astype(float)
## 幫Frequncy區分等級
new_customer_data_df['Frequency_Level'] = pd.cut(new_customer_data_df['Frequency'], bins=[1,2,3,4,5,10000000], labels=[1,2,3,4,5], right=False).astype(float)
## 幫Monetary區分等級
new_customer_data_df['Monetary_Level'] = pd.cut(new_customer_data_df['Monetary'], bins=[0,40,80,120,160,1000000000], labels=[1,2,3,4,5], right=False).astype(float)
## 顯示數據
new_customer_data_df

,Client_Id,Recency,Frequency,Monetary,Recency_Level,Frequency_Level,Monetary_Level
0,1,408,2,600.0,1.0,2.0,5.0
1,2,10,3,500.0,5.0,3.0,5.0
2,3,70,1,700.0,4.0,1.0,5.0
3,4,64,4,724.0,4.0,4.0,5.0
4,5,60,3,499.0,4.0,3.0,5.0
5,6,4,5,440.0,5.0,5.0,5.0
6,7,50,3,399.0,4.0,3.0,5.0
7,8,45,6,150.0,4.0,5.0,4.0
8,9,37,5,619.0,5.0,5.0,5.0
9,10,26,7,343.0,5.0,5.0,5.0


In [11]:
## 幫Recency區分等級
new_customer_data_df['Recency_Level'] = pd.cut(new_customer_data_df['Recency'], bins=[0,40,80,120,160,1000000], labels=[5,4,3,2,1], right=False).astype(float)
## 幫Frequncy區分等級
new_customer_data_df['Frequency_Level'] = pd.cut(new_customer_data_df['Frequency'], bins=[1,2,3,4,5,10000000], labels=[1,2,3,4,5], right=False).astype(float)
## 幫Monetary區分等級
new_customer_data_df['Monetary_Level'] = pd.cut(new_customer_data_df['Monetary'], bins=[0,40,80,120,160,1000000000], labels=[1,2,3,4,5], right=False).astype(float)
## 顯示數據
new_customer_data_df

,Client_Id,Recency,Frequency,Monetary,Recency_Level,Frequency_Level,Monetary_Level
0,1,408,2,600.0,1.0,2.0,5.0
1,2,10,3,500.0,5.0,3.0,5.0
2,3,70,1,700.0,4.0,1.0,5.0
3,4,64,4,724.0,4.0,4.0,5.0
4,5,60,3,499.0,4.0,3.0,5.0
5,6,4,5,440.0,5.0,5.0,5.0
6,7,50,3,399.0,4.0,3.0,5.0
7,8,45,6,150.0,4.0,5.0,4.0
8,9,37,5,619.0,5.0,5.0,5.0
9,10,26,7,343.0,5.0,5.0,5.0


In [12]:
new_customer_data_df['New_Recency'] = (new_customer_data_df['Recency_Level'] > new_customer_data_df['Recency_Level'].mean())*1

new_customer_data_df['New_Frequency'] = (new_customer_data_df['Frequency_Level'] > new_customer_data_df['Frequency_Level'].mean())*1

new_customer_data_df['New_Monetary'] = (new_customer_data_df['Monetary_Level'] > new_customer_data_df['Monetary_Level'].mean())*1

## 顯示數據
new_customer_data_df

,Client_Id,Recency,Frequency,Monetary,Recency_Level,Frequency_Level,Monetary_Level,New_Recency,New_Frequency,New_Monetary
0,1,408,2,600.0,1.0,2.0,5.0,0,0,1
1,2,10,3,500.0,5.0,3.0,5.0,1,0,1
2,3,70,1,700.0,4.0,1.0,5.0,0,0,1
3,4,64,4,724.0,4.0,4.0,5.0,0,1,1
4,5,60,3,499.0,4.0,3.0,5.0,0,0,1
5,6,4,5,440.0,5.0,5.0,5.0,1,1,1
6,7,50,3,399.0,4.0,3.0,5.0,0,0,1
7,8,45,6,150.0,4.0,5.0,4.0,0,1,0
8,9,37,5,619.0,5.0,5.0,5.0,1,1,1
9,10,26,7,343.0,5.0,5.0,5.0,1,1,1


In [13]:
## 組合欄位
new_customer_data_df['Customer_Group'] = new_customer_data_df['New_Recency']*100+ new_customer_data_df['New_Frequency']*10 + new_customer_data_df['New_Monetary']
## 顯示數據
new_customer_data_df

,Client_Id,Recency,Frequency,Monetary,Recency_Level,Frequency_Level,Monetary_Level,New_Recency,New_Frequency,New_Monetary,Customer_Group
0,1,408,2,600.0,1.0,2.0,5.0,0,0,1,1
1,2,10,3,500.0,5.0,3.0,5.0,1,0,1,101
2,3,70,1,700.0,4.0,1.0,5.0,0,0,1,1
3,4,64,4,724.0,4.0,4.0,5.0,0,1,1,11
4,5,60,3,499.0,4.0,3.0,5.0,0,0,1,1
5,6,4,5,440.0,5.0,5.0,5.0,1,1,1,111
6,7,50,3,399.0,4.0,3.0,5.0,0,0,1,1
7,8,45,6,150.0,4.0,5.0,4.0,0,1,0,10
8,9,37,5,619.0,5.0,5.0,5.0,1,1,1,111
9,10,26,7,343.0,5.0,5.0,5.0,1,1,1,111


In [14]:
## 組合欄位
new_customer_data_df['Customer_Group_value'] = new_customer_data_df['New_Recency']*100+ new_customer_data_df['New_Frequency']*10 + new_customer_data_df['New_Monetary']
## 顯示數據
new_customer_data_df
## 定義分群標準
## 定義分群標準
def customer_group(value):
    if value == 111:
        label = '最重要的顧客'
    elif value == 110:
        label = '未來有消費潛力的顧客'
    elif value == 101:
        label = '消費頻率較低的顧客'
    elif value == 100:
        label = '新顧客'
    elif value == 11:
        label = '近期流失的重要顧客'
    elif value == 10:
        label = '普通顧客'
    elif value == 1:
        label = '流失掉的高消費顧客'
    elif value == 0:
        label = '不是我們的客群'
     
    return label

## 將Gender 和 Age 合併回去
gender_df = customer_data_df.groupby('Client_Id')['Gender'].max().reset_index()
Age_df = customer_data_df.groupby('Client_Id')['Age'].max().reset_index()
new_customer_data_df = pd.merge(new_customer_data_df, gender_df, right_on="Client_Id", left_on="Client_Id")
new_customer_data_df = pd.merge(new_customer_data_df, Age_df, right_on="Client_Id", left_on="Client_Id")
 
## 顧客分群
new_customer_data_df['RFM_顧客分群結果'] = new_customer_data_df['Customer_Group_value'].apply(customer_group)

## 將數據保存為csv檔
new_customer_data_df.to_csv("RFM_Model_Result.csv", encoding="utf_8_sig")

## 顯示數據
new_customer_data_df



,Client_Id,Recency,Frequency,Monetary,Recency_Level,Frequency_Level,Monetary_Level,New_Recency,New_Frequency,New_Monetary,Customer_Group,Customer_Group_value,Gender,Age,RFM_顧客分群結果
0,1,408,2,600.0,1.0,2.0,5.0,0,0,1,1,1,M,26,流失掉的高消費顧客
1,2,10,3,500.0,5.0,3.0,5.0,1,0,1,101,101,F,32,消費頻率較低的顧客
2,3,70,1,700.0,4.0,1.0,5.0,0,0,1,1,1,M,40,流失掉的高消費顧客
3,4,64,4,724.0,4.0,4.0,5.0,0,1,1,11,11,F,22,近期流失的重要顧客
4,5,60,3,499.0,4.0,3.0,5.0,0,0,1,1,1,M,42,流失掉的高消費顧客
5,6,4,5,440.0,5.0,5.0,5.0,1,1,1,111,111,M,30,最重要的顧客
6,7,50,3,399.0,4.0,3.0,5.0,0,0,1,1,1,F,34,流失掉的高消費顧客
7,8,45,6,150.0,4.0,5.0,4.0,0,1,0,10,10,M,26,普通顧客
8,9,37,5,619.0,5.0,5.0,5.0,1,1,1,111,111,F,38,最重要的顧客
9,10,26,7,343.0,5.0,5.0,5.0,1,1,1,111,111,M,32,最重要的顧客
